In [148]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import sys
import itertools
import random
import torch
import os
import signal
import gc
import dgl
import networkx as nx
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras import backend as K
from keras.utils import plot_model
from numba import cuda,jit
import multiprocessing as mp
from multiprocessing import Pool, cpu_count
import psutil

np.set_printoptions(threshold=sys.maxsize,linewidth=512)
#np.set_printoptions(threshold=256)

from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))
 
#여백 줄이기
display(HTML("<style>.prompt { min-width: 10ex !important; }</style>"))

In [149]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    
    print(e)

In [150]:
target_length=100
unit_space_x=5
unit_space_y=5
unit_space = np.array([unit_space_y, unit_space_x])

In [151]:
def active_seed(matrix_size,density=0.8):
    active=np.random.choice(range(0,2),matrix_size,p=[density,1-density]).astype(np.int8)
    
    return active

        

In [152]:
def active_trans(seed,x,y):
    copy = seed.copy()
    for i in range(seed.shape[0]):
        for j in range(seed.shape[1]):
            if seed[i][j]==1:
                x_temp = np.random.choice(range(1,x+1))
                y_temp = np.random.choice(range(2,y+1))
                if j+x_temp > seed.shape[1]-1:
                    x_temp = seed.shape[1]-j
                if i+y_temp > seed.shape[0]-1:
                    y_temp = seed.shape[0]-i
                for a in range(y_temp):
                    for b in range(x_temp):
                        copy[i+a,j+b]=-1
                
    return copy

def gen_act(y,x,num=100):
    start=time.time()
    active_origin=np.zeros([1,y,x])
    density = np.array([0.6,0.7,0.8,0.8]) #pre-occupied active density
        
    for i in range(num):
        den = np.random.choice(density)
        seed=active_seed((y,x),den)
        active_origin=np.append(active_origin,active_trans(seed,2,4).reshape(1,y,x),axis=0)
    active_origin=np.delete(active_origin,0,axis=0)
    active_origin=active_origin.astype(np.int8)
    end=time.time()
    print(end-start)
    return active_origin

In [153]:
def active_trans(seed,x,y):
    copy = seed.copy()
    for i in range(seed.shape[0]):
        for j in range(seed.shape[1]):
            if seed[i][j]==1:
                x_temp = np.random.choice(range(1,x+1))
                y_temp = np.random.choice(range(2,y+1))
                if j+x_temp > seed.shape[1]-1:
                    x_temp = seed.shape[1]-j
                if i+y_temp > seed.shape[0]-1:
                    y_temp = seed.shape[0]-i
                for a in range(y_temp):
                    for b in range(x_temp):
                        copy[i+a,j+b]=-1
                
    return copy

def gen_act(y,x,num=100):
    start=time.time()
    active_origin = np.zeros([num, y, x], dtype=np.int8)
    density = np.array([0.6, 0.7, 0.8, 0.8]) #pre-occupied active density
    for i in range(num):
        den = np.random.choice(density)
        seed = active_seed((y,x),den)
        active_origin[i] = active_trans(seed,2,4)
    end=time.time()
    print(end-start)
    return active_origin

In [154]:
def random_normal_int(size):
    sigma=size/4
    m=size/2
    min=1
    max=size-1
    a=0
    while a <=0 or a>=size-1:
        a=math.floor(sigma*np.random.randn()+m)
    return a

In [155]:
def in_out_mid_gen(active_origin,target):
    min_rpt=0
    active = np.copy(active_origin)
    active_inout = np.copy(active_origin)
#   choice = np.random.choice(range(3))


#    x1 = np.random.choice(range(0,active_origin.shape[1]-(20*choice+20+1)))
#    x2 = np.random.choice(range(x1+(20*choice+20),active_origin.shape[1]))

    x1 = np.random.choice(range(0,active_origin.shape[1]-30-10*target-1))
    x2 = np.random.choice(range(x1+30+10*target,active_origin.shape[1]))

    count=0    
    for i in range(100000):
        y2 = np.random.choice(range(0,active_origin.shape[0]-2))
        y1 = np.random.choice(range(y2+1,active_origin.shape[0]))
        distance = abs(x1-x2)*unit_space_x+abs(y1-y2)*unit_space_y
#total target min distance >= 250um
        if distance >= target_length*2.5:
            min_rpt = int(distance/target_length)
            break
    if count>=10000:
        print("##########ERROR###########")
#    print("Input:",x1,y1)
#    print("Output:",x2,y2)
    mid = np.random.choice(range(x1+1,x2))
#    print("mid",mid)
    active[y1,x1]=2
    active[y2,x2]=2
    
    step1 = mid - x1
    step2 = abs(y2-y1)
    step3 = x2 - mid
    for i in range(step1):
        active[y1][x1+i+1]=1
    for i in range(step2+1):
        if y2-y1>=0:
            active[y1+i][mid]=1
        else:
            active[y2+i][mid]=1
    for i in range(step3):
        active[y2][mid+i+1]=1
    active[y1][x1]=2
    active[y2][x2]=2
    active_inout[y1][x1]=2
    active_inout[y2][x2]=2
    active_routing = np.copy(active)
    active_routing[active_routing==-1]=0
    return active,active_inout,active_routing,mid,distance,min_rpt,x1,x2,y1,y2


In [100]:
def find_start_end_mid(active_routing):
    a1=np.where(active_routing==1)[1]
    unique_elements, counts=np.unique(a1, return_counts=True)
    mid=unique_elements[counts>1][0]
    
    y1=np.where(active_routing==2)[0][1]
    y2=np.where(active_routing==2)[0][0]
    x1=np.where(active_routing==2)[1][1]
    x2=np.where(active_routing==2)[1][0]
    start= np.array([[y1,x1]])
    end = np.array([[y2,x2]])
    mid1 = np.array([[y1,mid]])
    mid2 = np.array([[y2,mid]])
    return start,end,mid1,mid2

In [101]:
def find_div_pos(divide,start,end,mid1,mid2):
    div_pos=np.copy(start)
    unit_space = np.array([unit_space_y, unit_space_x])
    line1_length = np.sum(np.multiply(abs(start[0]-mid1[0]), unit_space))
    line2_length = np.sum(np.multiply(abs(mid1[0]-mid2[0]), unit_space))
    line3_length = np.sum(np.multiply(abs(mid2[0]-end[0]), unit_space))
#    print(line1_length,line2_length,line3_length)
    distance = line1_length+line2_length+line3_length
    line_num=np.array([])
    for i in range(divide.shape[0]):
            
            if divide[i]<=line1_length :
                temp2 = np.copy(start)
                temp2[0][1] += np.round(divide[i]/unit_space_x)
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([1]))
            if line1_length<divide[i]<=line1_length+line2_length:
                temp2 = np.copy(mid1)
                temp2[0][0] -= np.round((divide[i]-line1_length)/unit_space_y)
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([2]))

            if divide[i] > line1_length+line2_length:
                temp2 = np.copy(mid2)
                temp2[0][1] += np.round((divide[i]-line1_length-line2_length)/unit_space_x)
                div_pos = np.append(div_pos,temp2,axis=0)
                line_num=np.append(line_num,np.array([3]))

                
    div_pos = np.delete(div_pos,0,axis=0)
    return div_pos,line_num,distance

In [102]:
def divide_line(active,min_rpt,start,end,mid1,mid2,add_num=0):

    min_rpt = min_rpt + add_num
    distance = abs(start-end)[0][0]*unit_space_y+abs(start-end)[0][1]*unit_space_x    
    unit_distance = int(distance / (min_rpt+1) )
    divide = np.array([])
    divide_line = np.array([])
    for i in range(min_rpt):
        divide = np.append(divide, np.array(int(unit_distance/unit_space_x)*unit_space_x*(i+1)))
        
    div_pos,line_num,distance = find_div_pos(divide,start,end,mid1,mid2)
    
    space=np.min(np.append(divide,distance)-np.append(0,divide))
    bound= int((target_length-space)/unit_space_x)        
    online_min=bound
    for i in range(div_pos.shape[0]-1):
        dist=int(math.dist(div_pos[i],div_pos[i+1]))
        if dist<online_min:
            online_min=dist
    return divide,div_pos,line_num,online_min,distance



In [103]:
def select_area(active,active_inout,divide,div_point,online_min,distance):

    space=np.min(np.append(divide,distance)-np.append(0,divide))
    bound= int((target_length-space)/max(unit_space_x,unit_space_y))

#    print(bound)
    x1=div_point[1]
    y1=div_point[0]
    a = np.array([[0,0]])
    
    if x1-bound < 0:
        
        b=np.arange(0,x1+bound+1)
    if (x1+bound) > (active.shape[1]-1):
        b=np.arange(x1-bound,active.shape[1])
    if x1-bound >= 0 and x1+bound <= active.shape[1]-1:
        b=np.arange(x1-bound,x1+bound+1)
        
    if y1-bound < 0:
        c=np.arange(0,y1+bound+1)
    if (y1+bound) > (active.shape[0]-1):
        c=np.arange(y1-bound,active.shape[0])
    if y1-bound >= 0 and y1+bound <= active.shape[0]-1:
        c=np.arange(y1-bound,y1+bound+1)   
    for i in range(c.shape[0]):
        for j in range(b.shape[0]):
            a=np.append(a,np.array([[c[i],b[j]]]),axis=0)
            
    a = np.delete(a,0,axis=0)
    c= np.array([[0, 0]])
    online=np.array([])
    for i in range(a.shape[0]):
        x2=a[i][1]
        y2=a[i][0]
        
        
        if active[y2][x2] == 0 and active_inout[y2][x2]==0:
            c=np.append(c,a[i].reshape(1,2),axis=0)
            online=np.append(online,0)
        if active[y2][x2] == 1 and active_inout[y2][x2]==0:
            if math.dist(div_point,a[i])<=online_min:
                c=np.append(c,a[i].reshape(1,2),axis=0)
                online=np.append(online,1)
            
    select= np.delete(c,0,axis=0)

    
    return select,online

In [104]:
def find_opt_delay(active,active_inout,divide,div_pos,online_min,distance):
    rpt_dist=np.array([])
    opt_pos=np.array([[0,0]])
    pos=np.copy(div_pos)
    min_dist = 10000
    error=0
    for i in range(div_pos.shape[0]):
        error=0
        select,online = select_area(active,active_inout,divide,div_pos[i],online_min,distance)
        if select.shape[0]==0:
            error=1
#            print("#######################NO ACTIVE################")
            break
        else:
            temp=abs(select-div_pos[i])
            min_d=np.argmin(np.sum(temp,axis=1))
            temp=np.sum(temp[min_d])
            if online[min_d]==1:
                pos[i]=select[min_d]
                temp=0
            opt_pos=np.append(opt_pos,select[min_d].reshape(1,2),axis=0)
            rpt_dist=np.append(rpt_dist,temp)
#            print(rpt_dist)
    opt_pos=np.delete(opt_pos,0,axis=0)
        
    
    return opt_pos,pos,rpt_dist,error

In [105]:
def cal_rpt_dist(active,opt_pos,pos,start,end,rpt_dist):

    temp = np.append(pos,end,axis=0)
    temp2 = np.append(start,pos,axis=0)
    part_dist = np.sum(abs(temp-temp2),axis=1)
#    print(part_dist)
    temp3 = np.append(rpt_dist,np.array(0))

    temp4 = np.append(np.array(0),rpt_dist)
    dist = (part_dist+temp3+temp4)*unit_space_x
    return dist  

In [107]:
def gen_example(y,x,num=1):
    start_time = time.time()
    input=np.zeros([1,y,x,3])
    output=np.zeros([1,y,x])

#    path1="./dataset3/input1k_"
#    path2="./dataset3/output1k_"
    path1="./dataset3/testset/input1k_"
    path2="./dataset3/testset/output1k_"
    
    check=num
    check2=0
    dist200=0
    dist300=0
    dist400=0    
    density = np.array([0.75,0.8,0.85,0.9]) #pre-occupied active density
    
    while num > 0 :
        
        if check-num == 1000:
            check2 +=1
            print("Generate 1K_"+str(check2))
            check-=1000
            input_part= np.delete(input,0,axis=0).astype(np.int8)
            output_part= np.delete(output,0,axis=0).astype(np.int8)
            np.save(path1+str(check2)+".npy",input_part)
            np.save(path2+str(check2)+".npy",output_part)
            input=np.zeros([1,y,x,3])
            output=np.zeros([1,y,x])            
            mid_time=time.time()
            print("time per 10k:",int(mid_time-start_time)," s")
        add_num=0
        error=0

        den = np.random.choice(density)
        seed=active_seed((y,x),den)        
        
        active_origin=active_trans(seed,2,4)
        
        seq = np.array([0,1,2,3])
        repeat_seq = np.tile(seq,num)[:num]
        
        active,active_inout,active_routing,mid,distance,min_rpt,x1,x2,y1,y2=in_out_mid_gen(active_origin,repeat_seq[num-1])
        start,end,mid1,mid2=find_start_end_mid(active_routing)


        for i in range(5):
            divide, div_pos,line_num,online_min,distance= divide_line(active,min_rpt,start,end,mid1,mid2,add_num)
            add_num+=1
            opt_pos,pos,rpt_dist,error=find_opt_delay(active,active_inout,divide,div_pos,online_min,distance)
            
            if error==1:
#                print("############RPT_ADD because NO ACTIVE")                
                continue
            dist=cal_rpt_dist(active,opt_pos,pos,start,end,rpt_dist)
            
           
            if np.max(dist)<=100:
                
                if 200<=distance<300:
                    dist200+=1
                elif 300<=distance<400:
                    dist300+=1
                elif distance>=400:
                    dist400+=1
#                print(dist)
#                print(np.sum(dist))
                num -= 1
                out=np.zeros(active.shape).astype(np.int8)
                for i in range(opt_pos.shape[0]):

                    xx=opt_pos[i][1]
                    yy=opt_pos[i][0]
                    out[yy][xx]=1
                out=out.reshape([1,y,x])

                temp=np.zeros([1,y,x,3])

                temp[0,:,:,0] = active.reshape(1,active.shape[0],active.shape[1])
                temp[0,:,:,1] = active_inout.reshape(1,active.shape[0],active.shape[1])
                temp[0,:,:,2] = active_routing.reshape(1,active.shape[0],active.shape[1])
                input=np.append(input,temp,axis=0)
                output = np.append(output,out,axis=0)
#                print("Add_num:::::",add_num)
                if add_num>=3:
                    print("$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$",add_num)
                
                break
            if add_num >= 5:
                if np.max(dist)>100:
                    error =1 
                    print("######################ERROR###############")
                

    output = np.delete(output,0,axis=0).astype(np.int8)
    input = np.delete(input,0,axis=0).astype(np.int8)
    np.save(path1+str(check2+1)+".npy",input)
    np.save(path2+str(check2+1)+".npy",output)
    end_time = time.time()
    print("Total time:",int(end_time-start_time),"s")
    print("dist200:",dist200)
    print("dist300:",dist300)
    print("dist400:",dist400)
    return input,output,dist
        


In [108]:
input,output,dist=gen_example(32,64,10000)

$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
Generate 1K_1
time per 10k: 17  s
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
Generate 1K_2
time per 10k: 36  s
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
Generate 1K_3
time per 10k: 55  s
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
Generate 1K_4
time per 10k: 74  s
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
$$$$$$$$$$$$$$$$$$$$$$$$$$MORE_THAN_3_ADD$$$$$$$$$$$$$ 3
Generate 

In [86]:
r_input=np.array([]).reshape
r_output=[]
for i in range(1,221):

        
    path1="./dataset3/input1k_"+str(i)+".npy"
    path2="./dataset3/output1k_"+str(i)+".npy"
    if i==1:
        r_input=np.load(path1)
        r_output=np.load(path2)
    else:
        r_input=np.append(r_input,np.load(path1),axis=0)
        r_output=np.append(r_output,np.load(path2),axis=0)
    

        
print(r_input.shape)
print(r_output.shape)

(220000, 32, 64, 3)
(220000, 32, 64)


In [87]:
#np.save("./dataset3/input220k.npy",r_input)
#np.save("./dataset3/output220k.npy",r_output)

In [89]:
r_input1=np.load("./dataset3/input280k.npy")
r_output1=np.load("./dataset3/output280k.npy")
r_input2=np.load("./dataset3/input220k.npy")
r_output2=np.load("./dataset3/output220k.npy")

r_input_combined = np.concatenate((r_input1, r_input2), axis=0)
r_output_combined = np.concatenate((r_output1, r_output2), axis=0)

print(r_input_combined.shape)
print(r_output_combined.shape)


(500000, 32, 64, 3)
(500000, 32, 64)


In [172]:
r_input=np.load("./dataset3/input500k.npy")
r_output=np.load("./dataset3/output500k.npy")
#r_input=np.load("./dataset3/testset/input1k_5.npy")
#r_output=np.load("./dataset3/testset/output1k_5.npy")


print(r_input.shape)
print(r_output.shape)

(1000, 32, 64, 3)
(1000, 32, 64)


In [173]:
r_output_modify = np.array([])
for i in range(r_output.shape[0]):
    r_output_modify=np.append(r_output_modify,np.sum(r_output[i]==1))

r_output_modify2=np.zeros([r_output.shape[0],10])
for i in range(r_output_modify.shape[0]):
                    temp=r_output_modify[i].astype(np.int32)
                    r_output_modify2[i][temp-1]=1

In [174]:
print(max(r_output_modify))
print(r_output_modify[5])
print(r_output_modify2[5])

6.0
3.0
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [175]:
#np.save("./dataset3/testset/output1k_5_num.npy",r_output_modify)
#np.save("./dataset3/testset/output1k_5_num_class.npy",r_output_modify2)


In [33]:
##Density Profile
density = np.array([])
total_num = 32*64
for i in range(r_input.shape[0]):
    density=np.append(density,np.count_nonzero(r_input[i,:,:,1])/total_num)
bin_edges = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
bin_indices = np.digitize(density, bin_edges)
bin_counts = np.bincount(bin_indices)
print(bin_counts)

[    0     0   213 66877 44998 58251 85701 23960]


In [ ]:
r_output_modify = np.array([])
for i in range(r_output.shape[0]):
    r_output_modify=np.append(r_output_modify,np.sum(r_output[i]==1))

In [15]:
#####tools for calculating distance to interconnect

def find_start_end_mid(active_routing):
    a1=np.where(active_routing==1)[1]
    unique_elements, counts=np.unique(a1, return_counts=True)
    mid=unique_elements[counts>1][0]
    
    y1=np.where(active_routing==2)[0][1]
    y2=np.where(active_routing==2)[0][0]
    x1=np.where(active_routing==2)[1][1]
    x2=np.where(active_routing==2)[1][0]
    start= np.array([[x1,y1]])
    end = np.array([[x2,y2]])
    mid1 = np.array([[mid,y1]])
    mid2 = np.array([[mid,y2]])
    return start,end,mid1,mid2

In [52]:
def cal_flat(active,y,x):
    x_dim = active.shape[1]
    y_dim = active.shape[0]
    coord = y*x_dim + x
    return coord

def cal_unflat(active,num):
    x_dim = active.shape[1]
    y_dim = active.shape[0]
    y = int(num / x_dim)
    x = num % x_dim
    return y,x

In [17]:
r_input=np.load("./gnn/dataset3/input500k.npy")
r_output=np.load("./gnn/dataset3/output500k_num.npy")
r_output2=np.load("./gnn/dataset3/output500k.npy")
rpt_num = np.argwhere(r_output==1)[:,1]+1


print(r_input.shape)
print(r_output.shape)
print(rpt_num.shape)

(500000, 32, 64, 3)
(500000, 10)
(500000,)


In [51]:
r_input.shape[3]

3

In [55]:
r_input_pos = np.copy(r_input)
r_output_pos = np.zeros([r_input_pos.shape[0],r_input_pos.shape[1]*r_input_pos.shape[2]])
for i in range(r_input_pos.shape[0]):
    active = r_input_pos[i,:,:,0]
    active_inout = r_input_pos[i,:,:,1]
    active_routing = r_input_pos[i,:,:,2]

    start,end,mid1,mid2 = find_start_end_mid(active_routing)
    divide, div_pos,line_num,online_min= divide_line(active,rpt_num[i],start,end,mid1,mid2,add_num=0)    
    
    for div in div_pos:
        active[div[::-1][0],div[::-1][1]]=4
        active_routing[div[::-1][0],div[::-1][1]]=4
        r_output_pos[i][cal_flat(active, div[::-1][0], div[::-1][1])]=1


In [73]:
np.save("./gnn/dataset3/input500k_pos.npy",r_input_pos)
np.save("./gnn/dataset3/output500k_pos.npy",r_output_pos.astype(np.int8))

In [75]:
r_input_pos = np.load("./gnn/dataset3/input500k_pos.npy")
r_output_pos = np.load("./gnn/dataset3/output500k_pos.npy")

print(r_input_pos.shape)
print(r_output_pos.shape)
print(r_output_pos.dtype)

(500000, 32, 64, 3)
(500000, 2048)
int8


(7, 46)